In [579]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, TimeDistributed, Dropout, AlphaDropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import *

In [580]:
tf.test.is_gpu_available()

True

In [581]:
train_data = pd.read_csv('data/task42.training.csv')
test_data = pd.read_csv('data/task42.test.WITHOUT.answers.csv')

In [582]:
train_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f16,f17,f18,f19,f20,f21,f22,f23,f24,class
0,27.247,5.235,21.148,1.819,18.794,2.827,4.132,10.042,14.238,15.316,...,0.571,-10.479,7.346,-16.383,-16.993,0.095,8.777,9.392,J,I
1,17.007,8.053,19.253,17.910,1.208,2.209,5.740,13.116,14.077,12.902,...,-10.122,-25.352,11.887,-7.805,-22.207,10.932,1.043,2.641,D,F
2,11.627,15.101,15.085,6.266,14.597,2.590,4.340,6.149,12.116,6.769,...,-9.695,-12.643,1.012,2.194,-10.821,9.390,0.328,8.167,D,B
3,7.848,12.820,6.811,2.360,16.185,3.721,1.859,30.002,7.198,19.888,...,-12.928,-18.363,2.534,-7.653,-10.458,0.939,1.643,9.689,I,G
4,6.060,11.139,8.253,6.030,15.321,3.572,4.840,29.867,8.839,19.805,...,6.736,3.794,1.009,6.736,-16.833,22.885,0.210,8.688,C,J


In [583]:
test_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,14.661,10.906,13.997,18.789,3.508,3.058,4.422,14.171,8.979,17.216,...,1.869,0.536,-14.941,12.178,-5.446,-2.566,0.977,3.530,6.853,J
1,14.578,7.126,21.684,12.571,13.452,2.394,3.902,9.056,13.781,13.020,...,1.769,-11.948,-21.154,5.587,-8.858,-18.778,6.249,0.946,8.471,G
2,15.877,9.065,12.778,12.995,15.872,1.113,3.961,15.509,10.669,12.952,...,-2.281,-22.332,-15.565,3.520,-9.337,-23.918,11.135,0.562,9.930,F
3,16.208,6.599,20.308,1.905,15.486,1.853,4.391,8.266,13.111,7.480,...,-0.414,0.741,-12.829,9.997,-10.729,-23.497,9.124,1.047,6.316,F
4,12.913,9.889,12.023,13.053,10.181,1.540,4.244,19.819,8.257,21.733,...,2.832,-9.252,-7.171,0.600,-9.252,-4.633,1.985,0.550,9.473,F


In [584]:
my_encoder = LabelEncoder()
# my_class_currency = np.array(my_encoder.fit_transform(train_data['f24'])).reshape(-1,1)
# #Create a "diccionary" to translate the categories into the actual values once you have the output
# my_class_decoder = list(np.unique(train_data['f24']))
process_array = my_encoder.fit_transform(train_data['f24'])
process_array_ans = my_encoder.fit_transform(train_data['class'])

In [585]:
df1_1p = pd.DataFrame(process_array, columns = ['PROC'])
train_data['f24'] = df1_1p['PROC']

In [586]:
X_train = np.array(train_data.iloc[:,0:25])
y_train = np.array(train_data.iloc[:,25:26]).reshape(-1)

In [587]:
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_npy = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = to_categorical(encoded_npy)

In [588]:
len(X_train)

30000

In [589]:
# tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

In [590]:
# create model
model = Sequential()

# =====STABLE=====
model.add(Dense(30, input_dim=25, activation='sigmoid'))
model.add(Dense(54, activation='swish'))
model.add(Dense(108, activation='swish'))
model.add(Dense(216, activation='tanh'))
model.add(Dense(108, activation='swish'))
model.add(Dense(54, activation='swish'))
model.add(Dense(10, activation='softmax'))
# ================

# model.add(LSTM(50, input_shape=(1,25) ,return_sequences=True))
# model.add(Dense(10, activation='softmax'))

# # Compile model
opt = Adam(learning_rate=0.0005)
model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_124"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_546 (Dense)            (None, 30)                780       
_________________________________________________________________
dense_547 (Dense)            (None, 54)                1674      
_________________________________________________________________
dense_548 (Dense)            (None, 108)               5940      
_________________________________________________________________
dense_549 (Dense)            (None, 216)               23544     
_________________________________________________________________
dense_550 (Dense)            (None, 108)               23436     
_________________________________________________________________
dense_551 (Dense)            (None, 54)                5886      
_________________________________________________________________
dense_552 (Dense)            (None, 10)             

In [591]:
model.fit(X_train, y_train, epochs=1000, batch_size=300, verbose=2, shuffle=True)

Epoch 1/1000
100/100 - 1s - loss: 1.8217 - accuracy: 0.2730
Epoch 2/1000
100/100 - 0s - loss: 1.3191 - accuracy: 0.4245
Epoch 3/1000
100/100 - 0s - loss: 1.1260 - accuracy: 0.5047
Epoch 4/1000
100/100 - 0s - loss: 1.0070 - accuracy: 0.5508
Epoch 5/1000
100/100 - 0s - loss: 0.9212 - accuracy: 0.5876
Epoch 6/1000
100/100 - 0s - loss: 0.8646 - accuracy: 0.6126
Epoch 7/1000
100/100 - 0s - loss: 0.8166 - accuracy: 0.6346
Epoch 8/1000
100/100 - 0s - loss: 0.7794 - accuracy: 0.6514
Epoch 9/1000
100/100 - 0s - loss: 0.7514 - accuracy: 0.6616
Epoch 10/1000
100/100 - 0s - loss: 0.7236 - accuracy: 0.6766
Epoch 11/1000
100/100 - 0s - loss: 0.7001 - accuracy: 0.6864
Epoch 12/1000
100/100 - 0s - loss: 0.6810 - accuracy: 0.6962
Epoch 13/1000
100/100 - 0s - loss: 0.6627 - accuracy: 0.7054
Epoch 14/1000
100/100 - 0s - loss: 0.6456 - accuracy: 0.7111
Epoch 15/1000
100/100 - 0s - loss: 0.6317 - accuracy: 0.7172
Epoch 16/1000
100/100 - 0s - loss: 0.6123 - accuracy: 0.7252
Epoch 17/1000
100/100 - 0s - loss

Epoch 135/1000
100/100 - 0s - loss: 0.0967 - accuracy: 0.9587
Epoch 136/1000
100/100 - 0s - loss: 0.1021 - accuracy: 0.9563
Epoch 137/1000
100/100 - 0s - loss: 0.0894 - accuracy: 0.9628
Epoch 138/1000
100/100 - 0s - loss: 0.0992 - accuracy: 0.9578
Epoch 139/1000
100/100 - 0s - loss: 0.1089 - accuracy: 0.9540
Epoch 140/1000
100/100 - 0s - loss: 0.0999 - accuracy: 0.9579
Epoch 141/1000
100/100 - 0s - loss: 0.0976 - accuracy: 0.9584
Epoch 142/1000
100/100 - 0s - loss: 0.1013 - accuracy: 0.9561
Epoch 143/1000
100/100 - 0s - loss: 0.0906 - accuracy: 0.9620
Epoch 144/1000
100/100 - 0s - loss: 0.0868 - accuracy: 0.9643
Epoch 145/1000
100/100 - 0s - loss: 0.0895 - accuracy: 0.9617
Epoch 146/1000
100/100 - 0s - loss: 0.1058 - accuracy: 0.9541
Epoch 147/1000
100/100 - 0s - loss: 0.0880 - accuracy: 0.9626
Epoch 148/1000
100/100 - 0s - loss: 0.0878 - accuracy: 0.9630
Epoch 149/1000
100/100 - 0s - loss: 0.1031 - accuracy: 0.9566
Epoch 150/1000
100/100 - 0s - loss: 0.0935 - accuracy: 0.9606
Epoch 15

100/100 - 0s - loss: 0.0767 - accuracy: 0.9668
Epoch 268/1000
100/100 - 0s - loss: 0.0620 - accuracy: 0.9733
Epoch 269/1000
100/100 - 0s - loss: 0.0624 - accuracy: 0.9727
Epoch 270/1000
100/100 - 0s - loss: 0.0722 - accuracy: 0.9696
Epoch 271/1000
100/100 - 0s - loss: 0.0717 - accuracy: 0.9698
Epoch 272/1000
100/100 - 0s - loss: 0.0681 - accuracy: 0.9708
Epoch 273/1000
100/100 - 0s - loss: 0.0603 - accuracy: 0.9737
Epoch 274/1000
100/100 - 0s - loss: 0.0638 - accuracy: 0.9733
Epoch 275/1000
100/100 - 0s - loss: 0.0640 - accuracy: 0.9727
Epoch 276/1000
100/100 - 0s - loss: 0.0839 - accuracy: 0.9633
Epoch 277/1000
100/100 - 0s - loss: 0.0606 - accuracy: 0.9755
Epoch 278/1000
100/100 - 0s - loss: 0.0647 - accuracy: 0.9727
Epoch 279/1000
100/100 - 0s - loss: 0.0689 - accuracy: 0.9704
Epoch 280/1000
100/100 - 0s - loss: 0.0606 - accuracy: 0.9755
Epoch 281/1000
100/100 - 0s - loss: 0.0667 - accuracy: 0.9722
Epoch 282/1000
100/100 - 0s - loss: 0.0909 - accuracy: 0.9622
Epoch 283/1000
100/100 

Epoch 400/1000
100/100 - 0s - loss: 0.0492 - accuracy: 0.9803
Epoch 401/1000
100/100 - 0s - loss: 0.0664 - accuracy: 0.9719
Epoch 402/1000
100/100 - 0s - loss: 0.0556 - accuracy: 0.9772
Epoch 403/1000
100/100 - 0s - loss: 0.0508 - accuracy: 0.9788
Epoch 404/1000
100/100 - 0s - loss: 0.0599 - accuracy: 0.9749
Epoch 405/1000
100/100 - 0s - loss: 0.0546 - accuracy: 0.9772
Epoch 406/1000
100/100 - 0s - loss: 0.0735 - accuracy: 0.9702
Epoch 407/1000
100/100 - 0s - loss: 0.0757 - accuracy: 0.9687
Epoch 408/1000
100/100 - 0s - loss: 0.0568 - accuracy: 0.9773
Epoch 409/1000
100/100 - 0s - loss: 0.0538 - accuracy: 0.9777
Epoch 410/1000
100/100 - 0s - loss: 0.0552 - accuracy: 0.9772
Epoch 411/1000
100/100 - 0s - loss: 0.0588 - accuracy: 0.9769
Epoch 412/1000
100/100 - 0s - loss: 0.0546 - accuracy: 0.9769
Epoch 413/1000
100/100 - 0s - loss: 0.0594 - accuracy: 0.9743
Epoch 414/1000
100/100 - 0s - loss: 0.0643 - accuracy: 0.9734
Epoch 415/1000
100/100 - 0s - loss: 0.0501 - accuracy: 0.9788
Epoch 41

100/100 - 0s - loss: 0.0450 - accuracy: 0.9809
Epoch 533/1000
100/100 - 0s - loss: 0.0876 - accuracy: 0.9649
Epoch 534/1000
100/100 - 0s - loss: 0.0508 - accuracy: 0.9782
Epoch 535/1000
100/100 - 0s - loss: 0.0526 - accuracy: 0.9777
Epoch 536/1000
100/100 - 0s - loss: 0.0531 - accuracy: 0.9781
Epoch 537/1000
100/100 - 0s - loss: 0.0463 - accuracy: 0.9809
Epoch 538/1000
100/100 - 0s - loss: 0.0564 - accuracy: 0.9756
Epoch 539/1000
100/100 - 0s - loss: 0.0928 - accuracy: 0.9645
Epoch 540/1000
100/100 - 0s - loss: 0.0449 - accuracy: 0.9813
Epoch 541/1000
100/100 - 0s - loss: 0.0442 - accuracy: 0.9819
Epoch 542/1000
100/100 - 0s - loss: 0.0440 - accuracy: 0.9814
Epoch 543/1000
100/100 - 0s - loss: 0.0494 - accuracy: 0.9801
Epoch 544/1000
100/100 - 0s - loss: 0.0546 - accuracy: 0.9760
Epoch 545/1000
100/100 - 0s - loss: 0.0516 - accuracy: 0.9780
Epoch 546/1000
100/100 - 0s - loss: 0.0560 - accuracy: 0.9757
Epoch 547/1000
100/100 - 0s - loss: 0.0566 - accuracy: 0.9758
Epoch 548/1000
100/100 

Epoch 665/1000
100/100 - 0s - loss: 0.0601 - accuracy: 0.9768
Epoch 666/1000
100/100 - 0s - loss: 0.0794 - accuracy: 0.9691
Epoch 667/1000
100/100 - 0s - loss: 0.0346 - accuracy: 0.9869
Epoch 668/1000
100/100 - 0s - loss: 0.0553 - accuracy: 0.9764
Epoch 669/1000
100/100 - 0s - loss: 0.0485 - accuracy: 0.9800
Epoch 670/1000
100/100 - 0s - loss: 0.0451 - accuracy: 0.9817
Epoch 671/1000
100/100 - 0s - loss: 0.0335 - accuracy: 0.9872
Epoch 672/1000
100/100 - 0s - loss: 0.0449 - accuracy: 0.9813
Epoch 673/1000
100/100 - 0s - loss: 0.0365 - accuracy: 0.9850
Epoch 674/1000
100/100 - 0s - loss: 0.0336 - accuracy: 0.9861
Epoch 675/1000
100/100 - 0s - loss: 0.0538 - accuracy: 0.9769
Epoch 676/1000
100/100 - 0s - loss: 0.0798 - accuracy: 0.9673
Epoch 677/1000
100/100 - 0s - loss: 0.0441 - accuracy: 0.9813
Epoch 678/1000
100/100 - 0s - loss: 0.0354 - accuracy: 0.9864
Epoch 679/1000
100/100 - 0s - loss: 0.0493 - accuracy: 0.9791
Epoch 680/1000
100/100 - 0s - loss: 0.0462 - accuracy: 0.9818
Epoch 68

100/100 - 0s - loss: 0.0380 - accuracy: 0.9841
Epoch 798/1000
100/100 - 0s - loss: 0.0418 - accuracy: 0.9831
Epoch 799/1000
100/100 - 0s - loss: 0.0470 - accuracy: 0.9810
Epoch 800/1000
100/100 - 0s - loss: 0.0482 - accuracy: 0.9805
Epoch 801/1000
100/100 - 0s - loss: 0.0554 - accuracy: 0.9790
Epoch 802/1000
100/100 - 0s - loss: 0.0319 - accuracy: 0.9877
Epoch 803/1000
100/100 - 0s - loss: 0.0444 - accuracy: 0.9828
Epoch 804/1000
100/100 - 0s - loss: 0.0585 - accuracy: 0.9784
Epoch 805/1000
100/100 - 0s - loss: 0.0293 - accuracy: 0.9889
Epoch 806/1000
100/100 - 0s - loss: 0.0535 - accuracy: 0.9784
Epoch 807/1000
100/100 - 0s - loss: 0.0514 - accuracy: 0.9798
Epoch 808/1000
100/100 - 0s - loss: 0.0351 - accuracy: 0.9860
Epoch 809/1000
100/100 - 0s - loss: 0.0311 - accuracy: 0.9879
Epoch 810/1000
100/100 - 0s - loss: 0.0405 - accuracy: 0.9838
Epoch 811/1000
100/100 - 0s - loss: 0.0418 - accuracy: 0.9827
Epoch 812/1000
100/100 - 0s - loss: 0.0831 - accuracy: 0.9692
Epoch 813/1000
100/100 

Epoch 930/1000
100/100 - 0s - loss: 0.0260 - accuracy: 0.9901
Epoch 931/1000
100/100 - 0s - loss: 0.0267 - accuracy: 0.9901
Epoch 932/1000
100/100 - 0s - loss: 0.0333 - accuracy: 0.9864
Epoch 933/1000
100/100 - 0s - loss: 0.0696 - accuracy: 0.9733
Epoch 934/1000
100/100 - 0s - loss: 0.0458 - accuracy: 0.9820
Epoch 935/1000
100/100 - 0s - loss: 0.0323 - accuracy: 0.9869
Epoch 936/1000
100/100 - 0s - loss: 0.0402 - accuracy: 0.9842
Epoch 937/1000
100/100 - 0s - loss: 0.0335 - accuracy: 0.9865
Epoch 938/1000
100/100 - 0s - loss: 0.0600 - accuracy: 0.9766
Epoch 939/1000
100/100 - 0s - loss: 0.0567 - accuracy: 0.9772
Epoch 940/1000
100/100 - 0s - loss: 0.0288 - accuracy: 0.9892
Epoch 941/1000
100/100 - 0s - loss: 0.0410 - accuracy: 0.9843
Epoch 942/1000
100/100 - 0s - loss: 0.0275 - accuracy: 0.9897
Epoch 943/1000
100/100 - 0s - loss: 0.0420 - accuracy: 0.9826
Epoch 944/1000
100/100 - 0s - loss: 0.0281 - accuracy: 0.9887
Epoch 945/1000
100/100 - 0s - loss: 0.0383 - accuracy: 0.9845
Epoch 94

In [592]:
process_test_array = my_encoder.transform(test_data['f24'])
df2 = pd.DataFrame(process_test_array, columns = ['PROC'])
test_data['f24'] = df2['PROC']
test_data

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,14.661000,10.906000,13.997000,18.789000,3.508000,3.058000,4.422000,14.171000,8.979000,17.216000,...,1.869000,0.536000,-14.941000,12.178000,-5.446000,-2.566000,0.977000,3.530000,6.853000,9
1,14.578000,7.126000,21.684000,12.571000,13.452000,2.394000,3.902000,9.056000,13.781000,13.020000,...,1.769000,-11.948000,-21.154000,5.587000,-8.858000,-18.778000,6.249000,0.946000,8.471000,6
2,15.877000,9.065000,12.778000,12.995000,15.872000,1.113000,3.961000,15.509000,10.669000,12.952000,...,-2.281000,-22.332000,-15.565000,3.520000,-9.337000,-23.918000,11.135000,0.562000,9.930000,5
3,16.208000,6.599000,20.308000,1.905000,15.486000,1.853000,4.391000,8.266000,13.111000,7.480000,...,-0.414000,0.741000,-12.829000,9.997000,-10.729000,-23.497000,9.124000,1.047000,6.316000,5
4,12.913000,9.889000,12.023000,13.053000,10.181000,1.540000,4.244000,19.819000,8.257000,21.733000,...,2.832000,-9.252000,-7.171000,0.600000,-9.252000,-4.633000,1.985000,0.550000,9.473000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,10.450070,13.886564,17.528795,14.845679,8.816005,0.445751,5.365857,16.053004,9.507735,21.967995,...,2.944044,-12.865365,-11.378178,0.362725,-3.909596,-14.788823,7.176737,0.224815,0.897055,0
5996,7.102688,11.264727,6.496310,15.737271,2.388523,3.873204,1.537952,6.964427,11.636381,6.593066,...,5.213355,0.883278,-1.235224,0.616699,-10.635310,-6.787539,1.509537,0.639168,8.369025,5
5997,25.576601,5.577068,21.840362,11.622796,12.879515,1.928998,4.370450,31.840880,4.984882,19.427728,...,1.708301,17.024050,-17.802395,41.104910,-4.499593,-12.286323,4.345729,3.075499,9.062645,9
5998,7.001098,13.092225,9.988685,17.710398,4.541493,3.312536,4.469947,9.158317,10.229346,6.035946,...,4.387957,-11.795072,-9.216711,0.828029,9.164179,-19.533970,30.747571,0.164103,6.996893,1


In [593]:
pred_y = model.predict(test_data)

In [594]:
pred_y[0:5]

array([[7.0241901e-26, 9.5124313e-23, 3.6294667e-18, 4.8161502e-13,
        9.9999809e-01, 1.8910035e-06, 2.3233571e-20, 2.8705532e-33,
        1.3970632e-21, 0.0000000e+00],
       [2.9574880e-20, 8.0322803e-24, 3.1252821e-18, 3.6158654e-23,
        8.5285007e-22, 2.0432711e-07, 9.9999976e-01, 3.6114672e-12,
        1.1513156e-11, 3.0199811e-31],
       [1.5251612e-22, 2.6409988e-22, 5.1566350e-17, 2.5091256e-20,
        2.2398411e-13, 9.9999976e-01, 2.5247664e-07, 9.1882734e-18,
        8.8194568e-14, 1.1364581e-33],
       [7.1011153e-21, 1.9975561e-24, 1.1979889e-17, 2.2858093e-25,
        4.4143485e-29, 4.7324118e-17, 9.9955779e-01, 4.4221830e-04,
        1.7445931e-10, 5.4586324e-28],
       [7.1456611e-21, 1.3194108e-22, 7.4640199e-17, 3.1078468e-18,
        6.1981966e-15, 9.9927312e-01, 7.2684838e-04, 9.8896211e-16,
        7.7649996e-12, 2.9857800e-32]], dtype=float32)

In [595]:
predicty = encoder.inverse_transform([np.argmax(item) for item in pred_y])
predicty

array(['E', 'G', 'F', ..., 'J', 'G', 'C'], dtype=object)

In [596]:
number = np.arange(1,len(predicty)+1)

In [597]:
sample = pd.read_csv('data/task42.SAMPLE.submission.csv')
sample.head()

,id,Predicted
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E


In [598]:
submission = {'Id':number, 'Predicted':predicty}
df = pd.DataFrame(submission,  columns= ['Id', 'Predicted'])

In [599]:
df.to_csv("submission.csv", index = False, header=True)